In [20]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
from MCForecastTools import MCSimulation
import json

%matplotlib inline

ModuleNotFoundError: No module named 'MCForecastTools'

In [10]:
?MCSimulation

Object `MCSimulation` not found.


In [2]:
load_dotenv()

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [3]:
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [4]:
today = pd.Timestamp("2020-06-01", tz="America/New_York").isoformat()


In [5]:
tickers = ["SPY", "AGG"]

In [6]:
timeframe = "1Day"

In [11]:
start_date = pd.Timestamp("1990-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()

In [12]:
# Get closing prices for SPY & AGG for past 31 years
df_portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

df_portfolio

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,108.54,108.8400,108.43,108.82,4259986,15562,108.754152,AGG
2015-12-02 05:00:00+00:00,108.73,108.7799,108.61,108.67,2462787,11581,108.684692,AGG
2015-12-03 05:00:00+00:00,108.41,108.4100,107.81,107.89,4634020,16801,108.040315,AGG
2015-12-04 05:00:00+00:00,108.05,108.3000,108.00,108.24,2182057,9796,108.192232,AGG
2015-12-07 05:00:00+00:00,108.30,108.5800,108.23,108.40,2143773,9104,108.460067,AGG
...,...,...,...,...,...,...,...,...
2021-12-27 05:00:00+00:00,472.09,477.3100,472.01,477.26,56689618,379424,475.278749,SPY
2021-12-28 05:00:00+00:00,477.80,478.8100,476.06,476.87,47190464,371806,477.232285,SPY
2021-12-29 05:00:00+00:00,477.04,478.5600,475.92,477.48,54405126,345000,477.260209,SPY


In [13]:
# Get closing prices for SPY & AGG for past 31 years

SPY = df_portfolio[df_portfolio['symbol']=='SPY'].drop('symbol', axis=1)
AGG = df_portfolio[df_portfolio['symbol']=='AGG'].drop('symbol', axis=1)

df_portfolio_year = pd.concat([SPY, AGG],axis=1, keys=['SPY','AGG'])

# set in ascending order (past to present)
df_portfolio_year = df_portfolio_year.sort_index()

df_portfolio_year.head()

SPY                                         \
                             open      high       low   close     volume   
timestamp                                                                  
2015-12-01 05:00:00+00:00  209.42  210.8200  209.1100  210.68   97858418   
2015-12-02 05:00:00+00:00  210.60  211.0000  208.2300  208.54  108069059   
2015-12-03 05:00:00+00:00  208.90  209.1500  204.7511  205.58  166224154   
2015-12-04 05:00:00+00:00  206.10  209.9700  205.9300  209.66  192878747   
2015-12-07 05:00:00+00:00  209.20  209.7295  207.2000  208.27  102027111   

                                                      AGG                    \
                          trade_count        vwap    open      high     low   
timestamp                                                                     
2015-12-01 05:00:00+00:00      337780  209.921410  108.54  108.8400  108.43   
2015-12-02 05:00:00+00:00      367013  209.563055  108.73  108.7799  108.61   
2015-12-03 05:00:00+00:00      546768  206.878936  108.41  108.4100  107.81   
2015-12-04 05:00:00+00:00      556731  208.178631  108.05  108.3000  108.00   
2015-12-07 05:00:00+00:00      374705  208.276128  108.30  108.5800  108.23   

                                                                    
                            close   volume trade_count        vwap  
timestamp                                                           
2015-12-01 05:00:00+00:00  108.82  4259986       15562  108.754152  
2015-12-02 05:00:00+00:00  108.67  2462787       11581  108.684692  
2015-12-03 05:00:00+00:00  107.89  4634020       16801  108.040315  
2015-12-04 05:00:00+00:00  108.24  2182057        9796  108.192232  
2015-12-07 05:00:00+00:00  108.40  2143773        9104  108.460067

In [14]:
daily_returns = df_portfolio_year.pct_change().dropna()
daily_returns.head()

SPY                                          \
                               open      high       low     close    volume   
timestamp                                                                     
2015-12-02 05:00:00+00:00  0.005635  0.000854 -0.004208 -0.010158  0.104341   
2015-12-03 05:00:00+00:00 -0.008072 -0.008768 -0.016707 -0.014194  0.538129   
2015-12-04 05:00:00+00:00 -0.013404  0.003921  0.005758  0.019846  0.160353   
2015-12-07 05:00:00+00:00  0.015041 -0.001145  0.006167 -0.006630 -0.471030   
2015-12-08 05:00:00+00:00 -0.013002 -0.006868 -0.006853 -0.006146  0.013185   

                                                      AGG                      \
                          trade_count      vwap      open      high       low   
timestamp                                                                       
2015-12-02 05:00:00+00:00    0.086544 -0.001707  0.001751 -0.000552  0.001660   
2015-12-03 05:00:00+00:00    0.489778 -0.012808 -0.002943 -0.003400 -0.007366   
2015-12-04 05:00:00+00:00    0.018222  0.006282 -0.003321 -0.001015  0.001762   
2015-12-07 05:00:00+00:00   -0.326955  0.000468  0.002314  0.002585  0.002130   
2015-12-08 05:00:00+00:00    0.034899 -0.006289  0.002678  0.000184  0.001294   

                                                                     
                              close    volume trade_count      vwap  
timestamp                                                            
2015-12-02 05:00:00+00:00 -0.001378 -0.421879   -0.255815 -0.000639  
2015-12-03 05:00:00+00:00 -0.007178  0.881616    0.450738 -0.005929  
2015-12-04 05:00:00+00:00  0.003244 -0.529122   -0.416939  0.001406  
2015-12-07 05:00:00+00:00  0.001478 -0.017545   -0.070641  0.002476  
2015-12-08 05:00:00+00:00  0.000369  0.078379    0.199912 -0.000057

In [15]:
# Create a simulation object
# This portfolio will have a 80/20 split between msft and aapl set in the weight parameter
# We set the number of simulations trials to be 100
# The period over which we will simulate is the number of trading days in a year times the number of years until the child reaches college.
# for this example, the child is 8 years old (meaning 10 years until college)
higher_risk_df = MCSimulation(
    portfolio_data=df_portfolio_year,
    weights=[0.80, 0.20],
    num_simulation=100,
    num_trading_days=252*10,
)

NameError: name 'MCSimulation' is not defined

In [18]:
higher_risk_df.portfolio_data.head()

NameError: name 'higher_risk_df' is not defined

In [19]:
# Cumulative Returns of higher risk portfolio
higher_risk_cumulative_returns = (1 + higher_risk_portfolio).cumprod()
higher_risk_cumulative_returns.head()

NameError: name 'higher_risk_portfolio' is not defined

In [12]:
# Set weights for low risk portfolio (20% SPY, 80% AGG)
low_risk_df = MCSimulation(
    portfolio_data=df_portfolio_year,
    weights=[0.20, 0.80],
    num_simulation=100,
    num_trading_days=252*10,
)

NameError: name 'MCSimulation' is not defined

In [13]:
low_risk_df.portfolio_data.head()

NameError: name 'low_risk_df' is not defined

In [17]:
# Cumulative Returns of low risk portfolio
low_risk_cumulative_returns = (1 + low_risk_portfolio).cumprod()
low_risk_cumulative_returns.head()

NameError: name 'low_risk_portfolio' is not defined

In [16]:
agg_daily_return = AGG.pct_change()
agg_daily_return

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-12-02 05:00:00+00:00,0.001751,-0.000552,0.001660,-0.001378,-0.421879,-0.255815,-0.000639
2015-12-03 05:00:00+00:00,-0.002943,-0.003400,-0.007366,-0.007178,0.881616,0.450738,-0.005929
2015-12-04 05:00:00+00:00,-0.003321,-0.001015,0.001762,0.003244,-0.529122,-0.416939,0.001406
2015-12-07 05:00:00+00:00,0.002314,0.002585,0.002130,0.001478,-0.017545,-0.070641,0.002476
...,...,...,...,...,...,...,...
2021-12-27 05:00:00+00:00,-0.001225,-0.000087,0.000526,0.000525,-0.012771,-0.098702,0.000701
2021-12-28 05:00:00+00:00,0.002453,0.001312,0.000350,-0.000438,-0.033580,0.056784,0.000453
2021-12-29 05:00:00+00:00,-0.003845,-0.003495,-0.003153,-0.002977,0.128607,0.111431,-0.003318


In [26]:
agg_cumulative_return = (1 + agg_daily_return).cumprod() - 1
agg_cumulative_return

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2020-06-01 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-02 04:00:00+00:00,0.000256,0.000597,0.000597,0.000852,-0.231745,-0.135220,0.000647
2020-06-03 04:00:00+00:00,-0.000170,0.000086,-0.001535,-0.001619,0.089171,0.035721,-0.001567
2020-06-04 04:00:00+00:00,-0.000682,-0.000851,-0.003582,-0.004601,-0.103473,-0.134168,-0.003312
2020-06-05 04:00:00+00:00,-0.005113,-0.004173,-0.006482,-0.004004,-0.074140,0.207665,-0.005334
